<a href="https://colab.research.google.com/github/akshaypt7/amazon_sales_prediction/blob/main/project_01_04_Using_entity_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction

In this part, we will use the embeddings created in the previous notebook and will train both random forest and neural network with the entity embedding.

#### Representation of Embeddings

This is how the embeddings are represneted for each category in the dataframe.


Embedding order = [asin_embedding, hours_embedding, dayofweek_embedding,
                  month_embedding, day_embedding, week_embedding]




{0: 0, 3: 1, 9: 2, 6: 3, 8: 4, 7: 5}




columns =   ['Asin', 'Item Promo Discount', 'price', 'hour', 'Invoice Amount',
       'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
       'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start',
       'Is_year_end', 'Is_year_start', 'Elapsed']




### Importing Libraries

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 5.2 MB/s 
     |████████████████████████████████| 188 kB 9.7 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 1.2 MB 11.4 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 51 kB 267 kB/s 
Mounted at /content/gdrive


In [ ]:
#hide
from fastbook import *
# from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
# from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder


In [ ]:
from sklearn.model_selection import ParameterGrid
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers import Dense, Flatten, Dropout, Input, Embedding, Reshape
from keras.layers import concatenate
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

from sklearn.model_selection import train_test_split
import math
import pandas as pd

In [ ]:
infile = open('/content/gdrive/MyDrive/bluebook/df_main.pkl','rb')
df_main = pickle.load(infile)

In [ ]:
df_main = add_datepart(df_main,'date')

In [ ]:
cat = ['Asin',
 'Is_month_end',
 'Is_month_start',
 'Is_quarter_end',
 'Is_quarter_start',
 'Is_year_end',
 'Is_year_start',
 'hour','Month','Week', 'Day','Dayofweek','Dayofyear']

In [ ]:
enc = OrdinalEncoder()
enc.fit(df_main[cat])
df_main[cat]= enc.transform(df_main[cat])

In [ ]:
condition = (df_main.Year < 2020) | (df_main.Month < 8)
train_idx = np.where(condition)[0]
valid_idx = np.where(~condition)[0]

In [ ]:
X =df_main.drop(['Quantity'],axis=1)

In [ ]:
X_train = df_main.drop(['Quantity'],axis=1).iloc[train_idx]
y_train = df_main['Quantity'].iloc[train_idx]

X_valid = df_main.drop(['Quantity'],axis=1).iloc[valid_idx]
y_valid = df_main['Quantity'].iloc[valid_idx]

In [ ]:
saved_embeddings_fname = '/content/gdrive/MyDrive/bluebook/embeddings.pickle'

### Loading the Embeddings

#### We are replacing the columns with thier respective entity embedding.

In [ ]:
def embed_features(X, saved_embeddings_fname):
    f_embeddings = open(saved_embeddings_fname, "rb")
    embeddings = pickle.load(f_embeddings)

    index_embedding_mapping = {0: 0, 3: 1, 9: 2, 6: 3, 8: 4, 7: 5}
    X_embedded = []

    (num_records, num_features) = X.shape
    for record in X:
        embedded_features = []
        for i, feat in enumerate(record):
            feat = int(feat)
            if i not in index_embedding_mapping.keys():
                embedded_features += [feat]
            else:
                embedding_index = index_embedding_mapping[i]
                embedded_features += embeddings[embedding_index][feat].tolist()

        X_embedded.append(embedded_features)

    return np.array(X_embedded)

In [ ]:
X_emb = embed_features(X.values,saved_embeddings_fname)

#### Using the same Valid and Train set we used earlier

In [ ]:
X_train = list(itemgetter(*train_idx)(X_emb)) 
X_valid = list(itemgetter(*valid_idx)(X_emb)) 

# or we can use list comprehension too for ex, test = [X_emb[i] for i in train_idx ]

In [ ]:

y_train = df_main['Quantity'].iloc[train_idx]

y_valid = df_main['Quantity'].iloc[valid_idx]

Note : We have created the train and test set from the X_embeddings. since X_embeddings is a list we use either list comprehension or list(itemgetter)

refer - https://stackoverflow.com/questions/25431850/passing-a-list-of-indices-to-another-list-in-python-correct-syntax